<a href="https://colab.research.google.com/github/mirj1209/AmazonV_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-01 12:51:07--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.26MB/s    in 0.2s    

2022-09-01 12:51:07 (6.26 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1774101|R3PMQXEVYR4ZT1|B00DVKOYBM|     263046246|         8 Ball Pool|     Mobile_Apps|          3|            0|          0|   N|                Y|                 Fun|It is a fun game ...| 2015-08-31|
|         US|   13679234|R1I1K3EO98EMUV|B00R1IA4RS|     128868854|Christmas Cookie ...|     Mobile_Apps|          5|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
vine_reviews_clean_df = df.dropna()

In [6]:
# Create the vine_table DataFrame
vine_df = vine_reviews_clean_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3PMQXEVYR4ZT1|          3|            0|          0|   N|                Y|
|R1I1K3EO98EMUV|          5|            0|          0|   N|                Y|
|R3K23URSI2MY42|          5|            4|          4|   N|                Y|
| RH2P770T10X38|          5|            0|          0|   N|                Y|
| RV9YBAW8CWFYN|          1|            0|          0|   N|                Y|
| R19627K7G1G6X|          5|            0|          0|   N|                Y|
|R22858XECEF7YQ|          3|            0|          0|   N|                N|
|R3KQOY7MDZOK0U|          1|            0|          0|   N|                Y|
|R31BKIN02QFDHS|          4|            0|          0|   N|                Y|
|R3FW62P5EUDUUV|          5|            0|          0|   N|     

In [8]:
# Filtered for more than 20 total votes
vine_filtered_votes_df = vine_df.filter("total_votes>=20")
vine_filtered_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1Z9QZRQJ1F8JX|          1|           17|         22|   N|                Y|
|R31PAYBVBP8W62|          1|           54|         81|   N|                Y|
|R33GCMAT1K5DX8|          2|          141|        190|   N|                Y|
|R3MIL83RY4SMS4|          1|           27|         38|   N|                Y|
|R37T9FXGG07XOM|          1|           24|         28|   N|                Y|
|R2E9ZHX72173PZ|          1|           14|         21|   N|                Y|
|R2I6YA0BVG78P6|          3|           59|         70|   N|                Y|
|R1NB53BPRH24RK|          5|           15|         26|   N|                Y|
|R3AX1CCCZH20C5|          5|           17|         26|   N|                Y|
|R1FOM4K4S3S1A3|          4|           19|         22|   N|     

In [9]:
# Filtered for more than 20 total votes, and more than 50% helpful
vine_helpful_by_total_df = vine_filtered_votes_df.filter("(helpful_votes/total_votes)>=0.5")
vine_helpful_by_total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1Z9QZRQJ1F8JX|          1|           17|         22|   N|                Y|
|R31PAYBVBP8W62|          1|           54|         81|   N|                Y|
|R33GCMAT1K5DX8|          2|          141|        190|   N|                Y|
|R3MIL83RY4SMS4|          1|           27|         38|   N|                Y|
|R37T9FXGG07XOM|          1|           24|         28|   N|                Y|
|R2E9ZHX72173PZ|          1|           14|         21|   N|                Y|
|R2I6YA0BVG78P6|          3|           59|         70|   N|                Y|
|R1NB53BPRH24RK|          5|           15|         26|   N|                Y|
|R3AX1CCCZH20C5|          5|           17|         26|   N|                Y|
|R1FOM4K4S3S1A3|          4|           19|         22|   N|     

In [10]:
# Filtered for paid reviews, more than 50% helpful
vine_paid_helpful_df = vine_helpful_by_total_df.filter("vine == 'Y'")
vine_paid_helpful_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [11]:
# Filtered for non-paid reviews, more than 50% helpful
vine_unpaid_helpful_df = vine_helpful_by_total_df.filter("vine == 'N'")
vine_unpaid_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1Z9QZRQJ1F8JX|          1|           17|         22|   N|                Y|
|R31PAYBVBP8W62|          1|           54|         81|   N|                Y|
|R33GCMAT1K5DX8|          2|          141|        190|   N|                Y|
|R3MIL83RY4SMS4|          1|           27|         38|   N|                Y|
|R37T9FXGG07XOM|          1|           24|         28|   N|                Y|
|R2E9ZHX72173PZ|          1|           14|         21|   N|                Y|
|R2I6YA0BVG78P6|          3|           59|         70|   N|                Y|
|R1NB53BPRH24RK|          5|           15|         26|   N|                Y|
|R3AX1CCCZH20C5|          5|           17|         26|   N|                Y|
|R1FOM4K4S3S1A3|          4|           19|         22|   N|     

In [12]:
# Total number of reviews
total_reviews_ct = vine_helpful_by_total_df.count()
total_reviews_ct

129508

In [13]:
# Total paid reviews
total_paid_reviews_df = vine_helpful_by_total_df.filter("vine =='Y'")
total_paid_reviews_ct = total_paid_reviews_df.count()
total_paid_reviews_ct

0

In [14]:
# Total unpaid reviews
total_unpaid_reviews_df = vine_helpful_by_total_df.filter("vine =='N'")
total_unpaid_reviews_ct = total_unpaid_reviews_df.count()
total_unpaid_reviews_ct

129508

In [15]:
# Total five-star reviews
total_five_star_reviews_df = vine_helpful_by_total_df.filter("star_rating == 5")
total_five_star_reviews_ct = total_five_star_reviews_df.count()
total_five_star_reviews_ct

59271

In [16]:
# Total five-star paid reviews
five_star_paid_reviews_df = total_paid_reviews_df.filter("star_rating == 5")
five_star_paid_reviews_ct = five_star_paid_reviews_df.count()
five_star_paid_reviews_ct

0

In [17]:
# Total five-star unpaid reviews
five_star_unpaid_reviews_df = total_unpaid_reviews_df.filter("star_rating < 5")
five_star_unpaid_reviews_ct = five_star_unpaid_reviews_df.count()
five_star_unpaid_reviews_ct

70237

In [18]:
# Five-star paid reviews as percent of total paid reviews
paid_five_star_per_total_paid = (five_star_paid_reviews_ct/total_paid_reviews_ct)*100
round(paid_five_star_per_total_paid, 3)

ZeroDivisionError: ignored

In [19]:
# Paid reviews as percent of total five-star reviews
paid_five_star_per_five_star_total = (five_star_paid_reviews_ct/total_five_star_reviews_ct)*100
round(paid_five_star_per_five_star_total, 3)

0.0

In [20]:
# Five-star unpaid reviews as percent of total unpaid reviews
unpaid_five_star_per_total_unpaid = (five_star_unpaid_reviews_ct/total_unpaid_reviews_ct)*100
round(unpaid_five_star_per_total_unpaid, 3)

54.234

In [21]:
# Five-star unpaid reviews as percent of total five-star reviews
unpaid_five_star_per_five_star_total = (five_star_unpaid_reviews_ct/total_five_star_reviews_ct)*100
round(unpaid_five_star_per_five_star_total, 3)

118.501